Cell 1: Setup & Basic Imports

In [1]:
# Mount Google Drive and install dependencies
from google.colab import drive
drive.mount('/content/drive')

!pip install -q transformers torch safetensors pytorch-crf

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForTokenClassification
from safetensors.torch import load_file
import json
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

drivepath = "/content/drive/MyDrive/intent_project"  # Change as needed


Mounted at /content/drive
Device: cpu


Cell 2: Define CRF Slot Filling Model Class

In [2]:
import torch.nn as nn
from transformers import AutoModel
from torchcrf import CRF

class XLMRobertaWithCRF(nn.Module):
    def __init__(self, model_name, num_labels, id2label, label2id):
        super().__init__()
        self.transformer = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.transformer.config.hidden_size, num_labels)
        self.crf = CRF(num_labels, batch_first=True)
        self.id2label = id2label
        self.label2id = label2id
        self.num_labels = num_labels

    def forward(self, input_ids, attention_mask, token_type_ids=None, labels=None):
        outputs = self.transformer(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )

        sequence_output = outputs[0]
        sequence_output = self.dropout(sequence_output)
        logits = self.classifier(sequence_output)

        if labels is not None:
            labels = torch.where(labels == -100, torch.zeros_like(labels), labels)
            loss = -self.crf(logits, labels, mask=attention_mask.bool(), reduction='mean')
            return {'loss': loss, 'logits': logits}
        else:
            predictions = self.crf.decode(logits, mask=attention_mask.bool())
            return {'logits': logits, 'predictions': predictions}

print("✓ CRF Slot Filling model class defined")


✓ CRF Slot Filling model class defined


Cell 3: Load Intent Classification Model

In [5]:
print("\n=== Loading Intent Classification Model ===")

drive_path = '/content/drive/MyDrive/intent_project'
intent_model_dir = f'{drive_path}/xlm-roberta-intent-classifier-final'

device = torch.device('cpu')

# Load without half precision and no device map (CPU friendly)
intent_tokenizer = AutoTokenizer.from_pretrained(intent_model_dir)
intent_model = AutoModelForSequenceClassification.from_pretrained(intent_model_dir)
intent_model.to(device)
intent_model.eval()

# Load intent mappings
with open(f'{intent_model_dir}/intent2id.json', 'r') as f:
    intent2id = json.load(f)
with open(f'{intent_model_dir}/id2intent.json', 'r') as f:
    id2intent = json.load(f)

print(f"✓ Intent model loaded ({len(id2intent)} intents)")



=== Loading Intent Classification Model ===
✓ Intent model loaded (60 intents)


Cell 4: Load CRF Slot Filling Model

In [6]:
print("\n=== Loading CRF Slot Filling Model ===")

slot_model_dir = f"{drive_path}/slot_filling_model_crf/final_model"

with open(f"{slot_model_dir}/id2label.json", 'r') as f:
    id2slot = json.load(f)
with open(f"{slot_model_dir}/label2id.json", 'r') as f:
    slot2id = json.load(f)

slot_tokenizer = AutoTokenizer.from_pretrained(slot_model_dir)

slot_model = XLMRobertaWithCRF(
    model_name="xlm-roberta-base",
    num_labels=len(slot2id),
    id2label=id2slot,
    label2id=slot2id
)

model_state = load_file(f"{slot_model_dir}/model.safetensors")
slot_model.load_state_dict(model_state)

slot_model.to(device)
slot_model.eval()

print(f"✓ Slot filling CRF model loaded ({len(slot2id)} labels)")



=== Loading CRF Slot Filling Model ===


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

✓ Slot filling CRF model loaded (111 labels)


Cell 5: Intent Inference Function

In [7]:
def predict_intent_utterance(tokenizer, model, id2label, device, utterance):
    inputs = tokenizer(
        utterance,
        return_tensors="pt",
        truncation=True,
        max_length=128,
        padding="max_length"
    ).to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=-1).cpu().numpy()[0]
        pred_id = logits.argmax(dim=-1).item()
        pred_label = id2label[str(pred_id)]
        confidence = probs[pred_id]
        return pred_label, confidence

Cell 6: CRF Slot Filling Inference Function

In [8]:
def extract_slots_utterance_crf(utterance, tokenizer, model, id2label, device):
    inputs = tokenizer(
        utterance.split(),
        is_split_into_words=True,
        return_tensors="pt",
        truncation=True,
        max_length=128,
        padding="max_length"
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask']
        )
        predictions = torch.tensor(outputs['predictions'][0])

    batch_encoding = tokenizer(
        utterance.split(),
        is_split_into_words=True,
        truncation=True,
        max_length=128,
        padding="max_length"
    )
    word_ids = batch_encoding.word_ids(batch_index=0)
    words = utterance.split()

    word_predictions = []
    for word_idx in range(len(words)):
        word_preds = []
        for token_idx, wid in enumerate(word_ids):
            if wid == word_idx:
                word_preds.append(predictions[token_idx])
        if word_preds:
            word_predictions.append(word_preds[0])

    slots = []
    current_slot_type = None
    current_slot_words = []
    for word, pred_id in zip(words, word_predictions):
        label = id2label[str(int(pred_id))]

        if label == 'O':
            if current_slot_type:
                slots.append({'type': current_slot_type, 'value': ' '.join(current_slot_words)})
            current_slot_type = None
            current_slot_words = []
        elif label.startswith('B-'):
            if current_slot_type:
                slots.append({'type': current_slot_type, 'value': ' '.join(current_slot_words)})
            current_slot_type = label[2:]
            current_slot_words = [word]
        elif label.startswith('I-'):
            slot_type = label[2:]
            if slot_type == current_slot_type:
                current_slot_words.append(word)
            else:
                if current_slot_type:
                    slots.append({'type': current_slot_type, 'value': ' '.join(current_slot_words)})
                current_slot_type = slot_type
                current_slot_words = [word]
    if current_slot_type:
        slots.append({'type': current_slot_type, 'value': ' '.join(current_slot_words)})

    return slots


Cell 7: Combined Inference

In [9]:
def process_utterance(utterance):
    intent, confidence = predict_intent_utterance(intent_tokenizer, intent_model, id2intent, device, utterance)
    slots = extract_slots_utterance_crf(utterance, slot_tokenizer, slot_model, id2slot, device)
    return {
        "utterance": utterance,
        "intent": intent,
        "confidence": confidence,
        "slots": slots
    }

Cell 8: Test Examples

In [10]:
test_utterances = [
    "wake me up at 6 AM tomorrow",
    "book a table at an Italian restaurant at 7pm",
    "play some jazz music by Miles Davis",
    "remind me to call John next Friday",
    "set alarm for 8 AM on Monday",
]

for i, utterance in enumerate(test_utterances, 1):
    print(f"\nUtterance {i}: {utterance}")
    result = process_utterance(utterance)

    print(f"  Intent: {result['intent']} (confidence: {result['confidence']:.2f})")
    if result['slots']:
        print("  Slots:")
        for slot in result['slots']:
            print(f"    - [{slot['type']}]: {slot['value']}")
    else:
        print("  Slots: None")



Utterance 1: wake me up at 6 AM tomorrow
  Intent: alarm_set (confidence: 1.00)
  Slots:
    - [time]: 6 AM
    - [date]: tomorrow

Utterance 2: book a table at an Italian restaurant at 7pm
  Intent: recommendation_locations (confidence: 0.79)
  Slots:
    - [food_type]: Italian
    - [business_type]: restaurant
    - [time]: 7pm

Utterance 3: play some jazz music by Miles Davis
  Intent: play_music (confidence: 1.00)
  Slots:
    - [music_genre]: jazz
    - [artist_name]: Miles Davis

Utterance 4: remind me to call John next Friday
  Intent: calendar_set (confidence: 1.00)
  Slots:
    - [event_name]: call John
    - [date]: Friday

Utterance 5: set alarm for 8 AM on Monday
  Intent: alarm_set (confidence: 1.00)
  Slots:
    - [time]: 8 AM
    - [date]: Monday


In [11]:
def interactive_predict():
    print("Enter 'exit' or 'quit' to stop.")
    while True:
        utterance = input("\nEnter your utterance: ").strip()
        if utterance.lower() in ['exit', 'quit']:
            print("Exiting interactive session.")
            break

        result = process_utterance(utterance)

        print(f"\nUtterance: {result['utterance']}")
        print(f"Intent: {result['intent']} (confidence: {result['confidence']:.2f})")

        if result['slots']:
            print("Slots:")
            for slot in result['slots']:
                print(f"  - [{slot['type']}]: {slot['value']}")
        else:
            print("Slots: None")

# Run interactive mode
interactive_predict()

Enter 'exit' or 'quit' to stop.

Enter your utterance: बेडरूम की लाइट चालू करें

Utterance: बेडरूम की लाइट चालू करें
Intent: iot_hue_lighton (confidence: 0.82)
Slots:
  - [house_place]: बेडरूम

Enter your utterance: రేపటికి క్యాబ్ బుక్ చేసుకోండి

Utterance: రేపటికి క్యాబ్ బుక్ చేసుకోండి
Intent: transport_taxi (confidence: 1.00)
Slots:
  - [date]: రేపటికి
  - [transport_type]: క్యాబ్

Enter your utterance: నాకు రేపు మీటింగ్ షెడ్యూల్ ఉందా?

Utterance: నాకు రేపు మీటింగ్ షెడ్యూల్ ఉందా?
Intent: calendar_query (confidence: 1.00)
Slots:
  - [date]: రేపు
  - [event_name]: మీటింగ్

Enter your utterance: quit
Exiting interactive session.
